In [1]:
import torch
from LLaVA.llava.model.builder import load_pretrained_model
from LLaVA.llava.mm_utils import get_model_name_from_path

from LLaVA.llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from LLaVA.llava.conversation import SeparatorStyle
from LLaVA.llava.model.builder import load_pretrained_model
from LLaVA.llava.utils import disable_torch_init
from LLaVA.llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
    KeywordsStoppingCriteria,
)

model_path = "checkpoints/llava-lora-1.8"
print(get_model_name_from_path(model_path))
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base="liuhaotian/llava-v1.5-7b",
    model_name=get_model_name_from_path(model_path)
)

/datadrive5/huypn16/anaconda3/envs/uasd/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2024-04-30 02:57:32,948] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
llava-lora-1.8
Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/datadrive5/huypn16/anaconda3/envs/uasd/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...


In [2]:
query = "Give me the sunglasses"
prompt = DEFAULT_IMAGE_TOKEN + "\n" + query

In [3]:

input_ids = (
    tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
    .unsqueeze(0)
    .cuda()
)

In [4]:
input_ids.shape

torch.Size([1, 12])

In [5]:
import requests
from PIL import Image
from io import BytesIO


def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image

def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out

images = load_images(["grasp-anything++/seen/image/0e73f01e5d4b1fc064f6ab381209891c376fa26bdc2dbf7578db9b611e6c6337.jpg"])
images_tensor = process_images(
    images,
    image_processor,
    model.config
).to(model.device, dtype=torch.float16)

In [12]:
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=images_tensor,
        do_sample=True,
        temperature=0.8,
        max_new_tokens=100,
        use_cache=True,
    )

In [13]:
output_ids

tensor([[    1,   301, 11259,  2927, 29889, 29871,     2]], device='cuda:0')

In [14]:
input_token_len = input_ids.shape[1]
outputs = tokenizer.batch_decode(
        output_ids, skip_special_tokens=True
    )[0]
outputs

'lenses color. '

In [ ]:
output_ids

In [ ]:
input_token_len = input_ids.shape[1]
outputs = tokenizer.batch_decode(
        output_ids[:, input_token_len:], skip_special_tokens=True
    )[0]
outputs

In [ ]:
from transformers import CLIPVisionModel, CLIPImageProcessor, CLIPVisionConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vision_tower_name = "openai/clip-vit-large-patch14-336"

processor = CLIPImageProcessor.from_pretrained(vision_tower_name)
vision_tower = CLIPVisionModel.from_pretrained(vision_tower_name, device_map="auto")
image_forward_out = vision_tower(images_tensor[0].to(device=device).unsqueeze(0), output_hidden_states=True)

In [ ]:
image_forward_out.last_hidden_state.shape